#### Q1 What's the version of BentoML you installed?

In [1]:
import bentoml
bentoml.__version__

'1.0.7'

#### Q2 How big approximately is the saved BentoML model?

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

In [3]:
df = pd.read_csv('CreditScoring.csv', delimiter=',', skipinitialspace=True)

In [4]:
df.columns = df.columns.str.lower()
status_values = {
    1:'ok',
    2:'default',
    0:'unk'
}
df = df.replace({'status' : status_values})


home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}
df = df.replace({'home' : home_values})

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df = df.replace({'material' : marital_values})

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df = df.replace({'records' : records_values})

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df = df.replace({'job' : job_values})

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [5]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [6]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

**Random forest**

In [7]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

**BentoML saving**

In [10]:
bentoml.sklearn.save_model('credit_risk_model', rf,
                          custom_objects={
                              "dictVectorizer" : dv
                          })

Model(tag="credit_risk_model:oe3ukscrhgfuzyrt", path="/Users/ekaterinakutovaia/bentoml/models/credit_risk_model/oe3ukscrhgfuzyrt/")

% bentoml models list  
**Tag**                        **Module**        **Size**  **Creation Time**         
credit_risk_model:gi67t6srfgf…  bentoml.sklearn  5.50 MiB  2022-10-21 12:14:49  
**5.5 MiB = 720 kb**

#### Q3 What would the pydantic class look like?

In [15]:
{
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}

{'name': 'Tim', 'age': 37, 'country': 'US', 'rating': 3.14}

In [16]:
class UserProfile(BaseModel):
    name: str
    age : int
    country: str
    rating: float

NameError: name 'BaseModel' is not defined

#### Q4 What version of scikit-learn was this model trained with?

bentoml models get mlzoomcamp_homework:qtzdz3slg6mwwdu5
name: mlzoomcamp_homework                                                                                                                                                                                   
version: qtzdz3slg6mwwdu5                                                                                                                                                                                   
module: bentoml.sklearn                                                                                                                                                                                     
labels: {}                                                                                                                                                                                                  
options: {}                                                                                                                                                                                                 
metadata: {}                                                                                                                                                                                                
context:                                                                                                                                                                                                    
  framework_name: sklearn                                                                                                                                                                                   
  framework_versions:                                                                                                                                                                                       
    **scikit-learn: 1.1.1**                                                                                                                                                                                     
  bentoml_version: 1.0.7                                                                                                                                                                                    
  python_version: 3.9.12                                                                                                                                                                                    
signatures:                                                                                                                                                                                                 
  predict:                                                                                                                                                                                                  
    batchable: false                                                                                                                                                                                        
api_version: v1                                                                                                                                                                                             
creation_time: '2022-10-13T20:42:14.411084+00:00'            

#### Q5 What value does model return? (input [[6.4,3.5,4.5,1.2]])

bentoml serve service5.py:svc --reload  

[[6.4,3.5,4.5,1.2]]  
  
**Response body: 1**